In [2]:
import sys
!{sys.executable} -m pip install pandas psycopg2

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 383 kB 1.5 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /Library/Developer/CommandLineTools/usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/w4/mky_skrx5fl9dqyyh_0fy5p40000gp/T/pip-install-z5vy8lj5/psycopg2_97670298fc254483b425c00b04588c5c/setup.py'"'"'; __file__='"'"'/private/var/folders/w4/mky_skrx5fl9dqyyh_0fy5p40000gp/T/pip-install-z5vy8lj5/psycopg2_97670298fc254483b425c00b04588c5c/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/w4/mky_skrx5fl9dqyyh_0fy5p40000gp/T/pip-pip-egg-info-ncqw42gb
         cwd: /private/var/

In [3]:
import pandas as pd

# create a database connection
import psycopg2
from sqlalchemy import create_engine

 

# Create an engine instance
alchemyEngine   = create_engine('postgresql+psycopg2://test:@127.0.0.1', pool_recycle=3600);

 

# Connect to PostgreSQL server
conn    = alchemyEngine.connect();

ModuleNotFoundError: No module named 'psycopg2'

In [12]:
# create the tables in the database
cursor = conn.cursor()

In [13]:
# read the CSV files into pandas dataframes
benefit_df = pd.read_csv('benefits.csv')
plans_df = pd.read_csv('plans.csv')
rates_df = pd.read_csv('rates.csv')

/var/folders/w4/mky_skrx5fl9dqyyh_0fy5p40000gp/T/ipykernel_48094/3850283623.py:2: DtypeWarning: Columns (9,12,19) have mixed types. Specify dtype option on import or set low_memory=False.
  benefit_df = pd.read_csv('benefits.csv')
/var/folders/w4/mky_skrx5fl9dqyyh_0fy5p40000gp/T/ipykernel_48094/3850283623.py:3: DtypeWarning: Columns (27,64,80,81,82,86,87,88,89,90,91,111,122,123,124,125,147,148) have mixed types. Specify dtype option on import or set low_memory=False.
  plans_df = pd.read_csv('plans.csv')


In [23]:
# Filter By 10 Most Populous States and Drop Unnecessary Columns
top10USStatesByPopulation = ['CA', 'TX', 'FL', 'NY', 'PA', 'IL', 'OH', 'GA', 'NC', 'MI']
# missing states = CA, NY, PA
benefit_df = benefit_df[benefit_df['StateCode'].isin(top10USStatesByPopulation)]
plans_df = plans_df[plans_df['StateCode'].isin(top10USStatesByPopulation)]
rates_df = rates_df[rates_df['StateCode'].isin(top10USStatesByPopulation)]

rates_df_filtered = rates_df.drop(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'ImportDate', 'RateEffectiveDate', 'RateExpirationDate', 'RatingAreaId'], axis=1)
plans_df_filtered = plans_df.drop(['BusinessYear', 'IssuerId', 'SourceName', 'ImportDate', 'PlanBrochure', 'URLForSummaryofBenefitsCoverage', 'HSAOrHRAEmployerContribution', 'HSAOrHRAEmployerContributionAmount', 'InpatientCopaymentMaximumDays', 'NationalNetwork', 'OutOfCountryCoverageDescription', 'OutOfServiceAreaCoverage', 'EHBPediatricDentalApportionmentQuantity', 'DesignType', 'MetalLevel'], axis=1)
benefit_df_filtered = benefit_df.drop(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'ImportDate', 'StandardComponentId', 'CopayInnTier1', 'CopayInnTier2', 'CopayOutofNet', 'CoinsInnTier1', 'CoinsInnTier2', 'CoinsOutofNet', 'Exclusions', 'Explanation', 'EHBVarReason', 'IsExclFromInnMOOP', 'IsExclFromOonMOOP'], axis=1)

In [24]:
# insert the data into the database
rates_df_filtered.to_sql('Rates', conn, if_exists='replace', index=False)
plans_df_filtered.to_sql('Plan', conn, if_exists='replace', index=False)
benefit_df_filtered.to_sql('Benefit', conn, if_exists='replace', index=False)

1113553